### Description:

This jupyter notebook will be testing to see how well models can predict a different calculated EPA. EPA in this file is calculated by subtracting the EP before the play and the EP after the play has finished.

In [17]:
import pandas as pd
import numpy as np


dataframe = pd.read_csv("big_ten_pbp.csv")

dataframe["soccer_time"] = dataframe["period"] * (
    16 - (dataframe["minutes"] + (dataframe["seconds"] / 60)).round(0)
).astype("int32")

soccer_time = range(1, 64, 3)
intervals = []
conditions = []

for i in soccer_time:
    if i > 60:
        intervals.append(f"{i}")
        conditions.append((dataframe["soccer_time"] == i))
    if i == 58:
        intervals.append(f"{i}-{i+2}")
        conditions.append(
            (dataframe["soccer_time"] >= i) & (dataframe["soccer_time"] <= (i + 2))
        )
    else:
        intervals.append(f"{i}-{i+3}")
        conditions.append(
            (dataframe["soccer_time"] >= i) & (dataframe["soccer_time"] <= (i + 3))
        )


dataframe["time_intervals"] = np.select(conditions, intervals)
dataframe["score_differential"] = (
    dataframe["offense_score"] - dataframe["defense_score"]
)
big_ten = dataframe[dataframe["offense_conference"] == "Big Ten"]
dataframe["time_intervals"].unique()

array(['1-4', '4-7', '7-10', '10-13', '13-16', '16-19', '25-28', '31-34',
       '37-40', '43-46', '46-49', '22-25', '34-37', '55-58', '61-64',
       '28-31', '19-22', '40-43', '49-52', '58-60', '0'], dtype=object)

In [18]:
field_pos_range = range(1, 100, 5)

intervals = []
conditions = []

for i in field_pos_range:
    intervals.append(f"{i}-{i+4}")
    conditions.append((big_ten["yard_line"] >= i) & (big_ten["yard_line"] <= (i + 4)))

big_ten["field_position_intervals"] = np.select(conditions, intervals)
big_ten["field_position_intervals"].unique()

/tmp/ipykernel_3868/3205648261.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  big_ten["field_position_intervals"] = np.select(conditions, intervals)


array(['31-35', '46-50', '51-55', '36-40', '21-25', '26-30', '41-45',
       '61-65', '66-70', '71-75', '81-85', '86-90', '91-95', '96-100',
       '16-20', '11-15', '0', '6-10', '56-60', '76-80', '1-5'],
      dtype=object)

In [19]:
conditions = [
    (big_ten["play_type"] == "Field Goal Missed"),
    (big_ten["play_type"] == "Field Goal Good"),
    (big_ten["play_type"] == "Passing Touchdown")
    | (big_ten["play_type"] == "Rusing Touchdown"),
    (big_ten["play_type"] == "Interception Return Touchdown")
    | (big_ten["play_type"] == "Fumble Return Touchdown")
    | (big_ten["play_type"] == "Punt Return Touchdown")
    | (big_ten["play_type"] == "Blocked Punt Touchdown")
    | (big_ten["play_type"] == "Missed Field Goal Return Touchdown")
    | (big_ten["play_type"] == "Blocked Field Goal Touchdown"),
    (big_ten["play_type"] == "Safety"),
]

values = [0, 3, 7, -7, 2]

big_ten["points_scored"] = np.select(conditions, values)

big_ten = big_ten[(big_ten["down"] > 0)]
big_ten = big_ten[(big_ten["distance"] > 0)]
big_ten = big_ten[(big_ten["time_intervals"] != "0")]
big_ten = big_ten[(big_ten["field_position_intervals"] != "0")]
print(big_ten["time_intervals"].unique())
print(big_ten["field_position_intervals"].unique())

big_ten[["time_lower", "time_upper"]] = big_ten.time_intervals.str.split(
    "-", expand=True
).astype("int")
big_ten[
    ["field_position_lower", "field_position_upper"]
] = big_ten.field_position_intervals.str.split("-", expand=True).astype("int")

big_ten["points_scored"].unique()

/tmp/ipykernel_3868/3404907034.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  big_ten["points_scored"] = np.select(conditions, values)


['1-4' '4-7' '7-10' '10-13' '13-16' '16-19' '25-28' '31-34' '43-46'
 '22-25' '34-37' '55-58' '28-31' '19-22' '40-43' '46-49' '49-52' '58-60'
 '37-40' '61-64']
['31-35' '46-50' '51-55' '36-40' '21-25' '26-30' '41-45' '61-65' '66-70'
 '71-75' '81-85' '86-90' '91-95' '96-100' '16-20' '11-15' '6-10' '56-60'
 '76-80' '1-5']


array([ 0,  7,  3, -7,  2])

In [20]:
grouped = (
    big_ten.groupby(
        [
            "down",
            "distance",
            "time_intervals",
            "field_position_intervals",
            "score_differential",
        ]
    )["points_scored"]
    .agg(["sum", "count", "mean", "median"])
    .reset_index()
)
grouped = grouped.astype(
    {"time_intervals": "category", "field_position_intervals": "category"}
)
big_ten = big_ten.astype(
    {"time_intervals": "category", "field_position_intervals": "category"}
)
grouped["mean"].unique()

array([ 0.        ,  7.        ,  3.5       ,  3.        ,  1.4       ,
        2.33333333,  2.        ,  2.8       ,  4.66666667,  1.75      ,
        0.73684211,  0.5       ,  0.20289855, -1.55555556,  0.65625   ,
        0.09395973,  0.19444444,  0.82352941,  1.16666667,  1.23529412,
        0.52830189,  0.7       , -0.13207547,  0.48275862,  0.05263158,
        0.77777778,  2.94736842,  1.27272727,  0.21875   ,  0.35897436,
        0.31111111,  0.23333333,  0.28      ,  0.60869565,  0.49122807,
        0.51219512,  1.        ,  0.53846154,  0.36842105,  0.93333333,
        0.46666667,  0.63636364,  0.58333333,  0.875     ,  1.07692308,
        0.56      ,  1.47368421,  0.4375    ,  1.10526316,  0.18421053,
        0.4516129 ,  0.42424242,  0.16666667, -2.33333333, -1.4       ,
        1.55555556,  1.86666667, -4.66666667, -7.        , -2.8       ,
        5.        ,  0.75      , -3.5       ,  1.5       , -1.16666667,
        0.4       ,  2.5       ,  0.51851852,  0.30434783,  0.15

In [21]:
intervals = []
conditions = []

big_ten[
    [
        "next_down",
        "next_distance",
        "next_time_interval",
        "next_field_position_interval",
        "next_score_differential",
        "next_home",
        "next_away",
    ]
] = big_ten[
    [
        "down",
        "distance",
        "time_intervals",
        "field_position_intervals",
        "score_differential",
        "home",
        "away",
    ]
].shift(
    -1
)

big_ten = big_ten[
    (big_ten["home"] == big_ten["next_home"])
    & (big_ten["away"] == big_ten["next_away"])
]

big_ten = big_ten[:-1]

i = 0
for row in grouped.to_dict("records"):
    if i % 10000 == 0:
        print(f"{i} completed")
    intervals.append(grouped["mean"][i])
    conditions.append(
        (big_ten["next_down"] == grouped["down"][i])
        & (big_ten["next_distance"] == grouped["distance"][i])
        & (big_ten["next_time_interval"] == grouped["time_intervals"][i])
        & (
            big_ten["next_field_position_interval"]
            == grouped["field_position_intervals"][i]
        )
        & (big_ten["next_score_differential"] == grouped["score_differential"][i])
    )
    i += 1

big_ten["xP After"] = np.select(conditions, intervals)
big_ten["xP After"].unique()

0 completed
10000 completed
20000 completed
30000 completed
40000 completed


array([ 0.        ,  0.15555556,  2.33333333,  0.23333333,  0.58333333,
        0.5       ,  0.52830189,  1.23529412,  0.7       ,  0.77777778,
        3.        ,  0.38888889,  0.1686747 ,  0.875     ,  4.66666667,
        1.75      ,  2.        ,  0.21538462,  3.5       , -0.13207547,
        0.28      ,  1.27272727,  0.75      ,  7.        ,  0.4       ,
       -7.        , -3.5       ,  0.36842105, -0.18666667,  0.73684211,
        1.16666667,  1.        ,  0.09395973,  0.2745098 ,  0.19178082,
        2.94736842, -0.31111111,  0.35897436,  0.65625   ,  0.21875   ,
        1.5       ,  0.49122807,  0.48275862,  0.51851852,  0.19444444,
        0.82352941,  0.05263158,  1.4       ,  0.56      ,  0.4516129 ,
       -1.55555556,  1.10526316,  1.86666667,  0.63636364,  0.93333333,
        1.55555556,  1.07692308,  2.8       ,  0.20289855,  0.25925926,
        0.18421053,  0.4375    , -1.4       , -2.33333333,  1.47368421,
       -1.16666667,  0.60869565,  0.46666667,  0.30434783,  0.42

In [22]:
# this cell is placing xP values in the right rows and then subtracting the points scored from the xp to get the xP added for the play.
# I wanted to see the average difference between my xPa model compared to the College football data's version of it
intervals = []
conditions = []

i = 0
for row in grouped.to_dict("records"):
    if i % 10000 == 0:
        print(f"{i} completed")
    intervals.append(row["mean"])
    conditions.append(
        (big_ten["down"] == row["down"])
        & (big_ten["distance"] == row["distance"])
        & (big_ten["time_intervals"] == row["time_intervals"])
        & (big_ten["field_position_intervals"] == row["field_position_intervals"])
        & (big_ten["score_differential"] == row["score_differential"])
    )
    i += 1

big_ten["xP Before"] = np.select(conditions, intervals)
big_ten["xP Before"].unique()

0 completed
10000 completed
20000 completed
30000 completed
40000 completed


array([ 0.        ,  0.15555556,  2.33333333,  0.23333333,  0.58333333,
        0.5       ,  0.52830189,  1.23529412,  0.7       ,  0.77777778,
        3.        ,  0.38888889,  0.1686747 ,  0.875     ,  4.66666667,
        1.75      ,  2.        ,  0.21538462,  3.5       , -0.13207547,
        0.28      ,  1.27272727,  0.75      ,  7.        ,  0.4       ,
       -7.        , -3.5       ,  0.36842105, -0.18666667,  0.73684211,
        1.16666667,  1.        ,  0.09395973,  0.2745098 ,  0.19178082,
        2.94736842, -0.31111111,  0.35897436,  0.65625   ,  0.21875   ,
        1.5       ,  0.49122807,  0.48275862,  0.51851852,  0.19444444,
        0.82352941,  0.05263158,  1.4       ,  0.56      ,  0.4516129 ,
       -1.55555556,  1.10526316,  1.86666667,  0.63636364,  0.93333333,
        1.55555556,  1.07692308,  2.8       ,  0.20289855,  0.25925926,
        0.18421053,  0.4375    , -1.4       , -2.33333333,  1.47368421,
       -1.16666667,  0.60869565,  0.46666667,  0.30434783,  0.42

In [23]:
# This cell is pulling and calculating the probability a first down is acheived by down and distance
# This will be used in the model making process to gauge how probable it is to make a first down

big_ten["xPa"] = big_ten["xP Before"] - big_ten["xP After"]

epa_diff = big_ten["xPa"] - big_ten["ppa"]
print(epa_diff.mean())

big_ten["got_first_down"] = big_ten["yards_gained"] >= big_ten["distance"]

first_down_prob = (
    big_ten.groupby(["down", "distance"])["got_first_down"]
    .agg(["sum", "count"])
    .reset_index()
)
first_down_prob["first_down_prob"] = first_down_prob["sum"] / first_down_prob["count"]

intervals = []
conditions = []

for i in range(len(first_down_prob)):
    intervals.append(first_down_prob["first_down_prob"][i])
    conditions.append(
        (big_ten["down"] == first_down_prob["down"][i])
        & (big_ten["distance"] == first_down_prob["distance"][i])
    )

big_ten["first_down_prob"] = np.select(conditions, intervals)

-0.16649782174631278


In [24]:
# Now let's create the models for the usual 4th down decision making. I will make 3 different models (Field Goals, Go For It, and Punting)

subset = big_ten[
    [
        "play_type",
        "down",
        "distance",
        "time_intervals",
        "field_position_intervals",
        "score_differential",
        "first_down_prob",
        "xPa",
    ]
]
subset = subset[~subset["play_type"].isin(["Kickoff", "Uncategorized"])]

fourth_down = subset[subset["down"] == 4]
fourth_down_dummies = pd.get_dummies(
    fourth_down, columns=["time_intervals", "field_position_intervals"]
)
predictiors = fourth_down_dummies.columns.drop(["xPa", "play_type"])
prediction_set = fourth_down_dummies[predictiors]

The models will be split up by the 3 decisions that could be made. One model for Field Goals, one for punting, and one for going for it.
I will then use each model to predict the xPa value for each 4th down situation for each deicison.

In [25]:
from sklearn.model_selection import train_test_split
from utils import rf_regress_params_tuner


field_goals_dummies = fourth_down_dummies[
    fourth_down_dummies["play_type"].isin(
        [
            "Field Goal Good",
            "Missed Field Goal Return Touchdown",
            "Missed Field Goal Return",
            "Blocked Field Goal",
            "Field Goal Missed",
            "Blocked Field Goal Touchdown",
        ]
    )
]

predictors = field_goals_dummies.columns.drop(
    ["xPa", "play_type"]
)  # predictor variables used: all variables besides the target variable
target = field_goals_dummies["xPa"].values  # target variable


# splits the subset into a training set to fit the models on and a testing set to test the models on for their accuracy
fg_train_data, fg_test_data, fg_train_sln, fg_test_sln = train_test_split(
    field_goals_dummies[predictors], target, test_size=0.2, random_state=0
)

field_goals_params = rf_regress_params_tuner(
    fg_train_data, fg_test_data, fg_train_sln, fg_test_sln, field_goals_dummies
)
print(field_goals_params)

INFO:root:Best r2 after tuning max depth is: 0.6695438390107427
INFO:root:Best r2 after tuning n_estimators is: 0.6762984706507735
INFO:root:Best r2 after tuning min samples split is: 0.6762984706507735
INFO:root:Best r2 after tuning min samples leaf is: 0.6762984706507735
INFO:root:Best r2 after tuning max features is: 0.6805983492162359
INFO:root:Final Model Accuracy 0.6805983492162359
INFO:root:Tuned parameters: {'max_depth': 29, 'n_estimators': 550, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 15}
INFO:root:Random Forest Feature Importance [0.         0.11567989 0.24201845 0.10819037 0.01381478 0.02453128
 0.01557169 0.01247991 0.02836428 0.01775617 0.01441972 0.01806129
 0.01981724 0.0088833  0.02024542 0.01645869 0.00447613 0.00851651
 0.01336843 0.00660206 0.00845228 0.00463022 0.00025511 0.02210449
 0.00823662 0.02121439 0.01225536 0.01467083 0.01808069 0.02300908
 0.00840218 0.00106267 0.         0.         0.         0.02514291
 0.01249893 0.01634182 0.02067

{'max_depth': 29, 'n_estimators': 550, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 15}


In [26]:
punts_dummies = fourth_down_dummies[
    fourth_down_dummies["play_type"].isin(
        ["Punt", "Blocked Punt", "Punt Return Touchdown", "Blocked Punt Touchdown"]
    )
]


predictors = punts_dummies.columns.drop(
    ["xPa", "play_type"]
)  # predictor variables used: all variables besides the target variable
target = punts_dummies["xPa"].values  # target variable


# splits the subset into a training set to fit the models on and a testing set to test the models on for their accuracy
punts_train_data, punts_test_data, punts_train_sln, punts_test_sln = train_test_split(
    punts_dummies[predictors], target, test_size=0.2, random_state=0
)

punts_params = rf_regress_params_tuner(
    punts_train_data, punts_test_data, punts_train_sln, punts_test_sln, punts_dummies
)

INFO:root:Best r2 after tuning max depth is: 0.3198968570332499
INFO:root:Best r2 after tuning n_estimators is: 0.3483045468943188
INFO:root:Best r2 after tuning min samples split is: 0.3483045468943188
INFO:root:Best r2 after tuning min samples leaf is: 0.3483045468943188
INFO:root:Best r2 after tuning max features is: 0.4004764817975298
INFO:root:Final Model Accuracy 0.4004764817975298
INFO:root:Tuned parameters: {'max_depth': 25, 'n_estimators': 600, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 14}
INFO:root:Random Forest Feature Importance [0.00000000e+00 1.68532530e-01 2.20158924e-01 1.73834691e-01
 2.02363735e-02 2.53302940e-02 2.15086226e-02 1.03062604e-02
 5.93433060e-03 3.74846981e-03 4.66642046e-02 2.77452687e-03
 1.17155721e-02 8.23173061e-03 8.42728775e-03 9.90713037e-03
 5.99900531e-05 1.06230131e-03 4.32501696e-03 3.35862791e-03
 4.54293516e-04 4.59943377e-03 2.96858093e-04 1.08194017e-02
 2.42409422e-05 2.13217902e-03 1.52905553e-02 1.12662848e-02
 2.03

In [27]:
go_for_it_dummies = fourth_down_dummies[
    ~fourth_down_dummies["play_type"].isin(
        [
            "Field Goal Good",
            "Missed Field Goal Return Touchdown",
            "Missed Field Goal Return",
            "Blocked Field Goal",
            "Field Goal Missed",
            "Blocked Field Goal Touchdown",
            "Punt",
            "Blocked Punt",
            "Punt Return Touchdown",
            "Blocked Punt Touchdown",
            "Penalty",
            "Timeout",
            "Kickoff Return (Offense)",
            "Kickoff Return Touchdown",
        ]
    )
]


predictors = go_for_it_dummies.columns.drop(
    ["xPa", "play_type"]
)  # predictor variables used: all variables besides the target variable
target = go_for_it_dummies["xPa"].values  # target variable


# splits the subset into a training set to fit the models on and a testing set to test the models on for their accuracy
(
    go_for_it_train_data,
    go_for_it_test_data,
    go_for_it_train_sln,
    go_for_it_test_sln,
) = train_test_split(
    go_for_it_dummies[predictors], target, test_size=0.2, random_state=0
)

go_for_it_params = rf_regress_params_tuner(
    go_for_it_train_data,
    go_for_it_test_data,
    go_for_it_train_sln,
    go_for_it_test_sln,
    field_goals_dummies,
)

INFO:root:Best r2 after tuning max depth is: 0.416104522399084
INFO:root:Best r2 after tuning n_estimators is: 0.4220282443782246
INFO:root:Best r2 after tuning min samples split is: 0.4220282443782246
INFO:root:Best r2 after tuning min samples leaf is: 0.4220282443782246
INFO:root:Best r2 after tuning max features is: 0.4677799175270059
INFO:root:Final Model Accuracy 0.4677799175270059
INFO:root:Tuned parameters: {'max_depth': 28, 'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 7}
INFO:root:Random Forest Feature Importance [0.         0.08807803 0.23554521 0.08393502 0.02002921 0.01264257
 0.02075255 0.01181143 0.01977832 0.01358436 0.01681339 0.01958677
 0.00619517 0.0151012  0.01272257 0.00354874 0.00346185 0.02250068
 0.00334721 0.00850124 0.01588289 0.02823174 0.02429115 0.00937581
 0.05949559 0.0032611  0.0095344  0.00123064 0.0188355  0.01351167
 0.00695757 0.01022477 0.00050825 0.00056584 0.02431629 0.00520124
 0.00193242 0.02349605 0.0068393

In [28]:
from sklearn.ensemble import RandomForestRegressor


fg_forest = RandomForestRegressor(
    max_depth=field_goals_params["max_depth"],
    n_estimators=field_goals_params["n_estimators"],
    min_samples_split=field_goals_params["min_samples_split"],
    min_samples_leaf=field_goals_params["min_samples_leaf"],
    max_features=field_goals_params["max_features"],
    random_state=0,
)

fg_forest.fit(fg_train_data, fg_train_sln)
predictions = fg_forest.predict(prediction_set)

fourth_down["field_goal_xPa"] = predictions
fourth_down.head()

/tmp/ipykernel_3868/2994112268.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fourth_down["field_goal_xPa"] = predictions


,play_type,down,distance,time_intervals,field_position_intervals,score_differential,first_down_prob,xPa,field_goal_xPa
10,Punt,4,3,1-4,51-55,0,0.409516,-0.155556,1.535591
33,Punt,4,6,7-10,26-30,7,0.342246,-0.233333,0.743420
64,Field Goal Missed,4,4,7-10,66-70,7,0.354616,0.000000,0.164554
105,Punt,4,7,1-4,21-25,7,0.314798,0.000000,1.317418
116,Punt,4,5,10-13,41-45,7,0.353859,0.000000,1.641415


In [29]:
punt_forest = RandomForestRegressor(
    max_depth=punts_params["max_depth"],
    n_estimators=punts_params["n_estimators"],
    min_samples_split=punts_params["min_samples_split"],
    min_samples_leaf=punts_params["min_samples_leaf"],
    max_features=punts_params["max_features"],
    random_state=0,
)

punt_forest.fit(punts_train_data, punts_train_sln)
predictions = punt_forest.predict(prediction_set)

fourth_down["punt_xPa"] = predictions

/tmp/ipykernel_3868/1977979914.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fourth_down["punt_xPa"] = predictions


In [30]:
gfi_forest = RandomForestRegressor(
    max_depth=go_for_it_params["max_depth"],
    n_estimators=go_for_it_params["n_estimators"],
    min_samples_split=go_for_it_params["min_samples_split"],
    min_samples_leaf=go_for_it_params["min_samples_leaf"],
    max_features=go_for_it_params["max_features"],
    random_state=0,
)

gfi_forest.fit(go_for_it_train_data, go_for_it_train_sln)
predictions = gfi_forest.predict(prediction_set)

fourth_down["go_for_it_xPa"] = predictions

/tmp/ipykernel_3868/1409615883.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fourth_down["go_for_it_xPa"] = predictions


In [31]:
fourth_down.head()

,play_type,down,distance,time_intervals,field_position_intervals,score_differential,first_down_prob,xPa,field_goal_xPa,punt_xPa,go_for_it_xPa
10,Punt,4,3,1-4,51-55,0,0.409516,-0.155556,1.535591,-0.014663,0.207349
33,Punt,4,6,7-10,26-30,7,0.342246,-0.233333,0.743420,-0.131027,0.068487
64,Field Goal Missed,4,4,7-10,66-70,7,0.354616,0.000000,0.164554,-0.147046,0.029694
105,Punt,4,7,1-4,21-25,7,0.314798,0.000000,1.317418,-0.015395,0.125685
116,Punt,4,5,10-13,41-45,7,0.353859,0.000000,1.641415,-0.004689,-0.057089


In [34]:
import openpyxl

conditions = [
    (fourth_down["field_goal_xPa"] > fourth_down["punt_xPa"])
    & (fourth_down["field_goal_xPa"] > fourth_down["go_for_it_xPa"]),
    (fourth_down["punt_xPa"] > fourth_down["field_goal_xPa"])
    & (fourth_down["punt_xPa"] > fourth_down["go_for_it_xPa"]),
    (fourth_down["go_for_it_xPa"] > fourth_down["field_goal_xPa"])
    & (fourth_down["go_for_it_xPa"] > fourth_down["punt_xPa"]),
]

values = ["Attempt FG", "Punt", "Go For It"]

fourth_down["suggested_decision"] = np.select(conditions, values)

fourth_down.to_excel("fourth_down_decision_making.xlsx", index=False)

/tmp/ipykernel_3868/3562247626.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fourth_down["suggested_decision"] = np.select(conditions, values)
